In [ ]:
# !pip install kafka-python

In [1]:
def count_and_send_back(key, value):
    import ast
    from pyspark import SparkContext
    from Kafka_Helpers import Producer
    import json
    
    value = value.replace("\'", "\"")
    o = json.loads(value)
    
    # Create Spark context
    sc = SparkContext.getOrCreate()
    
    # make list of lists out of string
    l = ast.literal_eval(str(o['reviews']))
    
    # flatten list of lists to one single list
    l = [item for sublist in l for item in sublist]
    
    # create Spark RDD from flattened list
    wordsRDD = sc.parallelize(l, 4)
    
    # count
    res = wordsRDD.map(lambda w: (w, 1)).reduceByKey(lambda x,y: x+y).collect()
    
    # send result (unique words + count) back to another Kafka topic
    p = Producer(server='kafka', port=9092)
    o['counted_words'] = res
    p.send('adjectives_counted', 'mykey', json.dumps(o))
    
from Kafka_Helpers import Consumer, Producer

Consumer(server='kafka', port=9092, topic_name='adjectives', handler=count_and_send_back)
# Consumer(server='kafka', port=9092, topic_name='topic_d', handler=print_key_value)
    

Waiting for new events...
